In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import bs4 as bs
import os
import numpy as np
import re
import time
from selenium.webdriver.common.keys import Keys
import nltk
import requests
import random
import math
import pathlib 
import modules.hein_scraping_functions

from modules.create_path import create_path
from modules.hein_scraping_functions import create_browser, webpage_wait, search_hein

In [2]:
# Create the paths for the data directories
input_path, work_path, intr_path, out_path, selenium_driver_path = create_path()

# Create the paths for the Chrome binary and selenium driver
chrome_binary_path = pathlib.Path("C:\\Program Files (x86)\\BraveSoftware\\Brave-Browser\\Application\\brave.exe")
selenium_driver_full_path = selenium_driver_path / "chromedriver.exe"

# Initalize the browsers that we are going to use
driver = create_browser(chrome_binary_path, selenium_driver_full_path)

driver.get("http://proxy.its.virginia.edu/login?url=http://heinonline.org/HOL/Welcome")


In [3]:
# Load the dataset from the input directory
book_df = pd.read_excel(input_path / "book_search.xlsx", sheet_name='for martin')

book_df

,Professor,Last,ID,num Coauthors,Title,search string1,search string2
0,"Adler, Matthew",Adler,1.0,1,Measuring Social Welfare: An Introduction,"""(MEASURING SOCIAL WELFARE) (ADLER)"" ~15","""(MEASURING SOCIAL WELFARE: AN INTRODUCTION) (..."
1,"Adler, Matthew",Adler,1.0,1,Well-Being And Fair Distribution: Beyond Cost-...,"""(WELL-BEING AND FAIR DISTRIBUTION) (ADLER)"" ~15","""(WELL-BEING AND FAIR DISTRIBUTION: BEYOND COS..."
2,"Adler, Matthew",Adler,1.0,1,New Foundations Of Cost-Benefit Analysis,"""(NEW FOUNDATIONS OF COST-BENEFIT ANALYSIS) (A...","""(NEW FOUNDATIONS OF COST-BENEFIT ANALYSIS) (A..."
3,"Albert, Richard",Albert,3.0,1,"Constitutional Amendments: Making, Breaking, A...","""(CONSTITUTIONAL AMENDMENTS) (ALBERT)"" ~15","""(CONSTITUTIONAL AMENDMENTS: MAKING, BREAKING,..."
4,"Boyd, William",Boyd,21.0,1,The Slain Wood: Papermaking And Its Environmen...,"""(THE SLAIN WOOD) (BOYD)"" ~15","""(THE SLAIN WOOD: PAPERMAKING AND ITS ENVIRONM..."
...,...,...,...,...,...,...,...
208,"Tomlins, Christopher",Tomlins,264.0,1,"Freedom, Bound: Law, Labor, And Civic Identity...","""(FREEDOM, BOUND) (TOMLINS)"" ~15","""(FREEDOM, BOUND: LAW, LABOR, AND CIVIC IDENTI..."
209,"Tomlins, Christopher",Tomlins,264.0,1,"Law, Labor, And Ideology In The Early American...","""(LAW, LABOR, AND IDEOLOGY IN THE EARLY AMERIC...","""(LAW, LABOR, AND IDEOLOGY IN THE EARLY AMERIC..."
210,"Tomlins, Christopher",Tomlins,264.0,1,"The State And The Unions: Labor Relations, Law...","""(THE STATE AND THE UNIONS) (TOMLINS)"" ~15","""(THE STATE AND THE UNIONS: LABOR RELATIONS, L..."
211,"Zellmer, Sandra",Zellmer,287.0,2,Mississippi River Tragedies: A Century Of Unna...,"""(MISSISSIPPI RIVER TRAGEDIES) (ZELLMER)"" ~15","""(MISSISSIPPI RIVER TRAGEDIES: A CENTURY OF UN..."


In [9]:
# Search for each string and save the results to the dataset
book_df_subset = book_df[:50]
book_df_subset["Results String1"] = book_df_subset["search string1"].apply(lambda x: search_hein(x, driver)).copy()
book_df_subset["Results String2"] = book_df_subset["search string2"].apply(lambda x: search_hein(x, driver)).copy()

NameError: name 'NoSuchElementException' is not defined

In [7]:
# Export to Excel
book_df_subset.to_excel(out_path / "_book_df.xlsx", index = False)